In [ ]:
import os
import base64
import requests
import pandas as pd
from io import StringIO
from google.colab import userdata

In [ ]:
api_key = userdata.get('API-key')

In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def explain_image_base64(question, base64_image):

    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

    payload = {
      "model": "gpt-4-vision-preview",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": question
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 1000
    }

    openai_url = "https://api.openai.com/v1/chat/completions"

    response = requests.post(openai_url, headers=headers, json=payload)
    return response.json()

def explain_image(question, image_path):
    base64_image = encode_image(image_path)
    return explain_image_base64(question, base64_image)

In [ ]:
image_path = r"Image.jpg"
json_answer = explain_image("""
Create csv file from this image that is a copy of English-German vocabulary. The csv file should contain 3 columns.
In the first column 'English word(s)' collect English words from the vocabulary.
In second column 'Transcription' place transcription. Use your knowledge to correct transcription if it is incorrect.
In the third column 'German Translations' collect transtaltions of the respective English word.
In the fourth column 'Examples' place a usage example of the respective English word. Please ignore examples from the other languages. Usually such examples are given for French with prefix Fr.
Be aware that some rows have several lines, especially in Examples - please combine it correctly based on gray line seperators.
User separator |. Be sure that csv has exactly 4 columns. Do not generate column titles row.
""", image_path)
answer = json_answer['choices'][0]['message']['content']
print(answer)

to declare | dɪˈkleər | erklären | In 1776 the 13 British colonies declared themselves independent from Britain.
general | ˈdʒenrəl | General | A general leads an army.
revolutionary | ˌrevəˈluːʃənəri | revolutionär | This idea is completely new – it's revolutionary!
peak | piːk | Haupt; Spitzen- | The peak immigration period was from 1880 to 1930.
immigration | ˌɪmɪˈɡreɪʃn | Immigration; Einwanderung; Einreise | The USA is a popular country for immigration.
a total of | ə ˈtəʊtl əv | insgesamt | For my vacation I paid a total of 500 dollars.
to pass | pɑːs | durchgehen; vorbeigehen (an), passieren | The train has just passed a bridge.
to represent | rɛprɪˈzɛnt | vertreten; darstellen; stehen für | The stars on the American flag represent the states.
covered wagon | ˈkʌvrd ˈwæɡən | Planwagen | They moved to the West in a covered wagon.
ancestor | ˈænsɛstər | Vorfahr/-in; Ahn/-in | My ancestors came from Scotland long ago.
nation | ˈneɪʃn | Nation | These houses were built in the 19th c

In [ ]:
lines = answer.split('\n')
lines

['to declare | dɪˈkleər | erklären | In 1776 the 13 British colonies declared themselves independent from Britain.',
 'general | ˈdʒenrəl | General | A general leads an army.',
 "revolutionary | ˌrevəˈluːʃənəri | revolutionär | This idea is completely new – it's revolutionary!",
 'peak | piːk | Haupt; Spitzen- | The peak immigration period was from 1880 to 1930.',
 'immigration | ˌɪmɪˈɡreɪʃn | Immigration; Einwanderung; Einreise | The USA is a popular country for immigration.',
 'a total of | ə ˈtəʊtl əv | insgesamt | For my vacation I paid a total of 500 dollars.',
 'to pass | pɑːs | durchgehen; vorbeigehen (an), passieren | The train has just passed a bridge.',
 'to represent | rɛprɪˈzɛnt | vertreten; darstellen; stehen für | The stars on the American flag represent the states.',
 'covered wagon | ˈkʌvrd ˈwæɡən | Planwagen | They moved to the West in a covered wagon.',
 'ancestor | ˈænsɛstər | Vorfahr/-in; Ahn/-in | My ancestors came from Scotland long ago.',
 'nation | ˈneɪʃn | Nati

In [ ]:
df = pd.read_csv(StringIO(answer), delimiter='|', names = ['E', 'T', 'D', 'Example'], header = 0)

In [ ]:
df

,E,T,D,Example
0,general,ˈdʒenrəl,General,A general leads an army.
1,revolutionary,ˌrevəˈluːʃənəri,revolutionär,This idea is completely new – it's revolution...
2,peak,piːk,Haupt; Spitzen-,The peak immigration period was from 1880 to ...
3,immigration,ˌɪmɪˈɡreɪʃn,Immigration; Einwanderung; Einreise,The USA is a popular country for immigration.
4,a total of,ə ˈtəʊtl əv,insgesamt,For my vacation I paid a total of 500 dollars.
5,to pass,pɑːs,"durchgehen; vorbeigehen (an), passieren",The train has just passed a bridge.
6,to represent,rɛprɪˈzɛnt,vertreten; darstellen; stehen für,The stars on the American flag represent the ...
7,covered wagon,ˈkʌvrd ˈwæɡən,Planwagen,They moved to the West in a covered wagon.
8,ancestor,ˈænsɛstər,Vorfahr/-in; Ahn/-in,My ancestors came from Scotland long ago.
9,nation,ˈneɪʃn,Nation,These houses were built in the 19th century.


In [ ]:
df['result'] = df.apply(lambda row:
    f'{row["E"]}<br>[{row["T"]}]<br><i><span style="color: rgb(170, 170, 0);">{row["Example"]}</span></i>|{row["D"]}',
    axis=1)



In [ ]:
result_str = '\n'.join(df['result'])

In [ ]:
print(result_str)

general <br>[ ˈdʒenrəl ]<br><i><span style="color: rgb(170, 170, 0);"> A general leads an army.</span></i>| General 
revolutionary <br>[ ˌrevəˈluːʃənəri ]<br><i><span style="color: rgb(170, 170, 0);"> This idea is completely new – it's revolutionary!</span></i>| revolutionär 
peak <br>[ piːk ]<br><i><span style="color: rgb(170, 170, 0);"> The peak immigration period was from 1880 to 1930.</span></i>| Haupt; Spitzen- 
immigration <br>[ ˌɪmɪˈɡreɪʃn ]<br><i><span style="color: rgb(170, 170, 0);"> The USA is a popular country for immigration.</span></i>| Immigration; Einwanderung; Einreise 
a total of <br>[ ə ˈtəʊtl əv ]<br><i><span style="color: rgb(170, 170, 0);"> For my vacation I paid a total of 500 dollars.</span></i>| insgesamt 
to pass <br>[ pɑːs ]<br><i><span style="color: rgb(170, 170, 0);"> The train has just passed a bridge.</span></i>| durchgehen; vorbeigehen (an), passieren 
to represent <br>[ rɛprɪˈzɛnt ]<br><i><span style="color: rgb(170, 170, 0);"> The stars on the American